# Installations

In [ ]:
!pip install transformers accelerate datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.8 MB/s eta 0:00:00


# Load dataset

In [ ]:
from datasets import load_dataset
data = load_dataset("csv", data_files= "/content/detoxification_dataset.csv", split="train[400000:500000]")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

Dataset({
    features: ['toxic', 'detoxified'],
    num_rows: 100000
})

In [ ]:
data = data.train_test_split(test_size=0.1)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['toxic', 'detoxified'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['toxic', 'detoxified'],
        num_rows: 10000
    })
})

# Model checkpoints

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Mapping the datasets (tokenization)

In [ ]:
prefix = "<dtox>"


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["toxic"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    labels = tokenizer(text_target=examples["detoxified"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_dataset = data["train"].map(preprocess_function, batched=True)
test_dataset = data["test"].map(preprocess_function, batched=True)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'toxic': '"Jesus fucking Christ," said Flynn.',
 'detoxified': "'jesus christ, 'said Flynn.",
 'input_ids': [3,
  2,
  26,
  235,
  226,
  3155,
  121,
  7851,
  7,
  302,
  3,
  89,
  4636,
  53,
  2144,
  976,
  243,
  8223,
  29,
  29,
  5,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [3,
  31,
  1924,
  7,
  302,
  3,
  15294,
  6,
  3,
  31,
  7,
  6146,
  8223,
  29,
  29,
  5,
  1]}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Model finetuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5-detoxification",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_steps = 100,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.161200,1.980335
2,2.081200,1.928030
3,2.083500,1.918718


TrainOutput(global_step=8439, training_loss=2.140083646587829, metrics={'train_runtime': 1301.877, 'train_samples_per_second': 207.393, 'train_steps_per_second': 6.482, 'total_flos': 3857520484417536.0, 'train_loss': 2.140083646587829, 'epoch': 3.0})